[View in Colaboratory](https://colab.research.google.com/github/margaretmz/fastai-notebooks/blob/master/keras_lesson1_colab_version.ipynb)

# Introduction to our first task: 'Dogs vs Cats'
**Updated to run in Google Colab by Margaret Maynard-Reid, 7/18/2018**

Original source code from the fast.ai repo [here](https://github.com/fastai/fastai/blob/master/courses/dl1/keras_lesson1.ipynb).

I have updated the notebook with the following:

* added data download code to make it work in codelab
* install the latest tensorflow version (v1.9.0)
* use tf.Keras which is part of the Tensorflow core API
* added comments to better explain the code

In [0]:
# Make sure we are using the lastest version of TensorFlow
!pip install -q -U tensorflow>=1.9.0 
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
# This line is needed only if using Colab - download the dogs cats datasets
!mkdir data && wget http://files.fast.ai/data/dogscats.zip && unzip dogscats.zip -d data/

mkdir: cannot create directory ‘data’: File exists


In [0]:
# Path where data is stored
PATH = "data/dogscats/"
# Image size
sz=224
# Batch size
batch_size=64

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras # Margaret: we are using the keras module from TensorFlow core API

print("TensorFlow verion is ", tf.__version__)
# from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing import image
# from keras.layers import Dropout, Flatten, Dense
# from keras.applications import ResNet50
# from keras.models import Model, Sequential
# from keras.layers import Dense, GlobalAveragePooling2D
# from keras import backend as K
# from keras.applications.resnet50 import preprocess_input

TensorFlow verion is  1.10.0-dev20180724


In [0]:
# Directory with training data
train_data_dir = f'{PATH}train'

# Directory with validation data
validation_data_dir = f'{PATH}valid'

## Data Augumentation

In [9]:
resnet50 = keras.applications.resnet50

train_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=resnet50.preprocess_input,
                                                             shear_range=0.2, 
                                                             zoom_range=0.2, 
                                                             horizontal_flip=True)

test_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(sz, sz),
                                                    batch_size=batch_size, 
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        shuffle=False,
                                                        target_size=(sz, sz),
                                                        batch_size=batch_size, class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Transfer learning

### Train only the last layer

In [10]:
# Define the base model from ResNet50
base_model = tf.keras.applications.ResNet50(weights='imagenet', 
                      include_top=False)
# Take a look at the base model architecture
# base_model.summary()

print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  174


In [0]:
# Let's take a look at the output of the base model
x = base_model.output

# Apply Global Average Pooling
x = keras.layers.GlobalAveragePooling2D()(x)

# Add a dense layer
x = keras.layers.Dense(1024, activation='relu')(x)

# Define the final output layer
predictions = keras.layers.Dense(1, activation='sigmoid')(x)

In [0]:
# Instantiate the model given the inputs and outputs
model = keras.models.Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model
for layer in base_model.layers: layer.trainable = False
  
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3


### Train more of the later layers

In [0]:
split_at = 170
# Freeze all the layers before split_at
for layer in model.layers[:split_at]: layer.trainable = False
  
# Unfreeze all the layers after aplit_at
for layer in model.layers[split_at:]: layer.trainable = True
  
# Compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])    

In [0]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1, workers=3,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/1
359/359 [==============================] - 348s 970ms/step - loss: 0.0148 - acc: 0.9957 - val_loss: 0.0681 - val_acc: 0.9879
CPU times: user 10min 46s, sys: 20.2 s, total: 11min 6s
Wall time: 5min 48s
